In [1]:
!conda install -y -c conda-forge xarray dask netCDF4 bottleneck

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.1
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [ ]:
!python -m pip install "xarray[complete]" 

In [9]:
import xarray as xr 
from src.utils.id_powerline_endpoints import load_counties, load_states
import geopandas as gpd
from shapely import Point
import pandas as pd 

import numpy as np
from shapely import contains_xy

In [3]:
# df = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2022-wb13-6h-0p25deg-chunk-1.zarr-v2")
df = xr.open_zarr("gs://weatherbench2/datasets/era5/1959-2022-6h-64x33.zarr")

In [ ]:
df.to_dataframe()

In [4]:
filtered_ds = df.sel(time=slice('2021-01-01', None))
names = list(filtered_ds.coords)
filtered_ds = filtered_ds.assign_coords(longitude=xr.where(filtered_ds.longitude > 180,
                                                             filtered_ds.longitude - 360,
                                                             filtered_ds.longitude))

counties = load_counties()
# Create a single geometry for the entire United States by merging the counties
us_geom = counties.unary_union

# Create a 2D grid of longitude and latitude coordinates from filtered_ds
# (filtered_ds dimensions: latitude then longitude)
lon_grid, lat_grid = np.meshgrid(filtered_ds.longitude, filtered_ds.latitude)

# Create a boolean mask where each grid point is inside the US geometry
mask = contains(us_geom, lon_grid, lat_grid)

margin_degrees = 10 / 69  # approx. 10 miles in degrees
buffered_us = us_geom.buffer(margin_degrees)

lon_grid, lat_grid = np.meshgrid(filtered_ds.longitude, filtered_ds.latitude)
mask = contains(buffered_us, lon_grid, lat_grid)

mask_da = xr.DataArray(mask, dims=("latitude", "longitude"),
                            coords={"latitude": filtered_ds.latitude, "longitude": filtered_ds.longitude})
filtered_ds = filtered_ds.where(mask_da, drop=True)
print(filtered_ds)

/tmp/ipykernel_84270/1898769529.py:9: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  us_geom = counties.unary_union
/tmp/ipykernel_84270/1898769529.py:16: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask = contains(us_geom, lon_grid, lat_grid)


<xarray.Dataset> Size: 61MB
Dimensions:                                           (time: 1460,
                                                       longitude: 16,
                                                       latitude: 6, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 48B ...
  * level                                             (level) int64 104B 50 ....
  * time                                              (time) datetime64[ns] 12kB ...
  * longitude                                         (longitude) float64 128B ...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 561kB dask.array<chunksize=(16, 16, 6), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 561kB dask.array<chunksize=(16, 16, 6), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 561kB

/tmp/ipykernel_84270/1898769529.py:22: DeprecationWarning: The 'shapely.vectorized.contains' function is deprecated and will be removed a future version. Use 'shapely.contains_xy' instead (available since shapely 2.0.0).
  mask = contains(buffered_us, lon_grid, lat_grid)


In [10]:
# Aggregate weather data for each state: average values over all grid nodes
# falling within the state geometry. Fallback to interpolation (centroid) if no grid node falls inside.

# load_states() and filtered_ds are already defined.
states = load_states().to_crs("EPSG:4326")
states["centroid"] = states.geometry.centroid

state_weather = {}  # dictionary to hold aggregated/interpolated weather data

# Compute meshgrid from filtered_ds coordinates.
lon_vals = filtered_ds.longitude.values
lat_vals = filtered_ds.latitude.values
lon_mesh, lat_mesh = np.meshgrid(lon_vals, lat_vals)

for idx, state in states.iterrows():
    # Compute mask for grid nodes that lie inside the state's geometry
    mask_state = contains_xy(state.geometry, lon_mesh, lat_mesh)
    
    if mask_state.any():
        # Create an xarray DataArray for the mask with the same dims as filtered_ds spatial coords.
        da_mask = xr.DataArray(mask_state, dims=("latitude", "longitude"),
                                 coords={"latitude": lat_vals, "longitude": lon_vals})
        # Subset filtered_ds to grid points inside the state
        state_subset = filtered_ds.where(da_mask, drop=True)
        # Aggregate (e.g., average) over latitude and longitude dimensions.
        state_data = state_subset
    else:
        # Fallback: use the centroid to look up the nearest grid cell.
        print(state["NAME"], "has no grid points inside its geometry. Using centroid for interpolation.")
        # centroid = state["centroid"]
        # nearest_lat = filtered_ds.latitude.sel(latitude=centroid.y, method="nearest")
        # nearest_lon = filtered_ds.longitude.sel(longitude=centroid.x, method="nearest")
        # state_data = filtered_ds.sel(latitude=nearest_lat, longitude=nearest_lon, method="nearest")
        state_data = None
    state_weather[state["NAME"]] = state_data

# Optionally, inspect the aggregated/interpolated weather data for each state
for state_name, data in state_weather.items():
    print(f"Weather data for {state_name}:")
    print(data)

/tmp/ipykernel_84270/3073346170.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  states["centroid"] = states.geometry.centroid


Nebraska has no grid points inside its geometry. Using centroid for interpolation.
Washington has no grid points inside its geometry. Using centroid for interpolation.
Kentucky has no grid points inside its geometry. Using centroid for interpolation.
Alabama has no grid points inside its geometry. Using centroid for interpolation.
Pennsylvania has no grid points inside its geometry. Using centroid for interpolation.
Missouri has no grid points inside its geometry. Using centroid for interpolation.
North Carolina has no grid points inside its geometry. Using centroid for interpolation.
Oklahoma has no grid points inside its geometry. Using centroid for interpolation.
New York has no grid points inside its geometry. Using centroid for interpolation.
Indiana has no grid points inside its geometry. Using centroid for interpolation.
Idaho has no grid points inside its geometry. Using centroid for interpolation.
North Dakota has no grid points inside its geometry. Using centroid for interpol

In [11]:
state_weather["Utah"].compute().to_dataframe()

10m_u_component_of_wind  \
time                longitude latitude level                            
2021-01-01 00:00:00 -112.5    39.375   50                         NaN   
                                       100                        NaN   
                                       150                        NaN   
                                       200                        NaN   
                                       250                        NaN   
...                                                               ...   
2021-12-31 18:00:00 -112.5    39.375   600                        NaN   
                                       700                        NaN   
                                       850                        NaN   
                                       925                        NaN   
                                       1000                       NaN   

                                              10m_v_component_of_wind  \
time                longitude latitude level                            
2021-01-01 00:00:00 -112.5    39.375   50                         NaN   
                                       100                        NaN   
                                       150                        NaN   
                                       200                        NaN   
                                       250                        NaN   
...                                                               ...   
2021-12-31 18:00:00 -112.5    39.375   600                        NaN   
                                       700                        NaN   
                                       850                        NaN   
                                       925                        NaN   
                                       1000                       NaN   

                                              10m_wind_speed  2m_temperature  \
time                longitude latitude level                                   
2021-01-01 00:00:00 -112.5    39.375   50                NaN             NaN   
                                       100               NaN             NaN   
                                       150               NaN             NaN   
                                       200               NaN             NaN   
                                       250               NaN             NaN   
...                                                      ...             ...   
2021-12-31 18:00:00 -112.5    39.375   600               NaN             NaN   
                                       700               NaN             NaN   
                                       850               NaN             NaN   
                                       925               NaN             NaN   
                                       1000              NaN             NaN   

                                              angle_of_sub_gridscale_orography  \
time                longitude latitude level                                     
2021-01-01 00:00:00 -112.5    39.375   50                                  NaN   
                                       100                                 NaN   
                                       150                                 NaN   
                                       200                                 NaN   
                                       250                                 NaN   
...                                                                        ...   
2021-12-31 18:00:00 -112.5    39.375   600                                 NaN   
                                       700                                 NaN   
                                       850                                 NaN   
                                       925                                 NaN   
                                       1000                                NaN   

                                      

In [22]:
state_weather["Texas"]["10m_u_component_of_wind"]

<xarray.DataArray '10m_u_component_of_wind' (time: 1460, longitude: 2,
                                             latitude: 1)> Size: 12kB
dask.array<where, shape=(1460, 2, 1), dtype=float32, chunksize=(100, 2, 1), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 8B 33.75
  * time       (time) datetime64[ns] 12kB 2021-01-01 ... 2021-12-31T18:00:00
  * longitude  (longitude) float64 16B -101.2 -95.62
Attributes:
    long_name:   10 metre U wind component
    short_name:  u10
    units:       m s**-1

In [23]:
filtered_ds.to_dataframe()

10m_u_component_of_wind  \
time                longitude latitude level                            
2021-01-01 00:00:00 -163.125  33.75    50                         NaN   
                                       100                        NaN   
                                       150                        NaN   
                                       200                        NaN   
                                       250                        NaN   
...                                                               ...   
2021-12-31 18:00:00 -67.500   67.50    600                        NaN   
                                       700                        NaN   
                                       850                        NaN   
                                       925                        NaN   
                                       1000                       NaN   

                                              10m_v_component_of_wind  \
time                longitude latitude level                            
2021-01-01 00:00:00 -163.125  33.75    50                         NaN   
                                       100                        NaN   
                                       150                        NaN   
                                       200                        NaN   
                                       250                        NaN   
...                                                               ...   
2021-12-31 18:00:00 -67.500   67.50    600                        NaN   
                                       700                        NaN   
                                       850                        NaN   
                                       925                        NaN   
                                       1000                       NaN   

                                              10m_wind_speed  2m_temperature  \
time                longitude latitude level                                   
2021-01-01 00:00:00 -163.125  33.75    50                NaN             NaN   
                                       100               NaN             NaN   
                                       150               NaN             NaN   
                                       200               NaN             NaN   
                                       250               NaN             NaN   
...                                                      ...             ...   
2021-12-31 18:00:00 -67.500   67.50    600               NaN             NaN   
                                       700               NaN             NaN   
                                       850               NaN             NaN   
                                       925               NaN             NaN   
                                       1000              NaN             NaN   

                                              angle_of_sub_gridscale_orography  \
time                longitude latitude level                                     
2021-01-01 00:00:00 -163.125  33.75    50                                  NaN   
                                       100                                 NaN   
                                       150                                 NaN   
                                       200                                 NaN   
                                       250                                 NaN   
...                                                                        ...   
2021-12-31 18:00:00 -67.500   67.50    600                                 NaN   
                                       700                                 NaN   
                                       850                                 NaN   
                                       925                                 NaN   
                                       1000                                NaN   

                                      